In [20]:
import pandas as pd

from sklearn.model_selection import train_test_split

all_data = pd.read_csv("https://tinyurl.com/y2qmhfsr", delimiter=",")

# Extraer las columnas de entrada, escalar por 255
all_inputs = all_data.iloc[:, 0:3].values
all_outputs = all_data.iloc[:, -1].values

# Dividir en un conjunto de entrenamiento y uno de prueba
X_train, X_test, Y_train, Y_test = train_test_split(all_inputs, all_outputs,
    test_size=1/3)

n = X_train.shape[0]
print(n)

print(all_inputs)
print("------------------")
print(all_outputs)

896
[[  0   0   0]
 [  0   0 128]
 [  0   0 139]
 ...
 [ 97  97  97]
 [ 99 184 255]
 [ 99  99  99]]
------------------
[0 0 0 ... 0 1 0]


In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

all_data = pd.read_csv("heart_statlog.csv")

# Extraer las columnas de entrada
all_inputs = all_data.iloc[:, 0:11].values
all_outputs = all_data.iloc[:, -1].values

# Pruebas para verificar los datos extraídos
""" n = X_train.shape[0]
print(n)
print(all_inputs)
print("------------------")
print(all_outputs) """

# Dividir en un conjunto de entrenamiento y uno de prueba
X_train, X_test, Y_train, Y_test = train_test_split(all_inputs, all_outputs, test_size=1/3)
n = X_train.shape[0]

# La tasa de aprendizaje
L = 0.05  

# Construir una red neuronal con pesos y sesgos
# iniciados aleatoriamente

# 11 neuronas ocultas con 11 pesos cada una (un peso por cada entrada)
# 1 neurona de salida con 11 pesos cada una (un peso por cada neurona oculta)
w_hidden = np.random.rand(11, 11)
w_output = np.random.rand(1, 11)

#1 sesgo para cada una de las 11 neuronas ocultas
#1 sesgo para la neurona de salida
b_hidden = np.random.rand(11, 1)
b_output = np.random.rand(1, 1)

# Funciones de activacion
relu = lambda x: np.maximum(x, 0)
logistic = lambda x: 1 / (1 + np.exp(-x))

# Ejecutar entradas a través de la red neuronal para obtener salidas predichas
def forward_prop(X):
    Z1 = w_hidden @ X + b_hidden
    A1 = relu(Z1)
    Z2 = w_output @ A1 + b_output
    A2 = logistic(Z2)
    return Z1, A1, Z2, A2

# Derivadas de las funciones de activación
d_relu = lambda x: x > 0
d_logistic = lambda x: np.exp(-x) / (1 + np.exp(-x)) ** 2

# Devuelve pendientes para pesos y sesgos
# usando la regla de la cadena
def backward_prop(Z1, A1, Z2, A2, X, Y):
    dC_dA2 = 2 * A2 - 2 * Y
    dA2_dZ2 = d_logistic(Z2)
    dZ2_dA1 = w_output
    dZ2_dW2 = A1
    dZ2_dB2 = 1
    dA1_dZ1 = d_relu(Z1)
    dZ1_dW1 = X
    dZ1_dB1 = 1

    dC_dW2 = dC_dA2 @ dA2_dZ2 @ dZ2_dW2.T

    dC_dB2 = dC_dA2 @ dA2_dZ2 * dZ2_dB2

    dC_dA1 = dC_dA2 @ dA2_dZ2 @ dZ2_dA1

    dC_dW1 = dC_dA1 @ dA1_dZ1 @ dZ1_dW1.T

    dC_dB1 = dC_dA1 @ dA1_dZ1 * dZ1_dB1

    return dC_dW1, dC_dB1, dC_dW2, dC_dB2

# Ejecutar descenso de gradiente
for i in range(100_000):
    # seleccionar aleatoriamente uno de los datos de entrenamiento
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx]

    # pasar datos seleccionados aleatoriamente a través de la red neuronal
    Z1, A1, Z2, A2 = forward_prop(X_sample)

    # distribuir error a través de la retropropagación
    # y devolver pendientes para pesos y sesgos
    dW1, dB1, dW2, dB2 = backward_prop(Z1, A1, Z2, A2, X_sample, Y_sample)

    # actualizar pesos y sesgos
    w_hidden -= L * dW1
    b_hidden -= L * dB1
    w_output -= L * dW2
    b_output -= L * dB2

# Interactuar y probar con nuevos conjuntos de síntomas
def predict_probability(age, sex, chest_pain_type, resting_bps, cholesterol, fasting_blood_sugar, resting_ecg, max_heart_rate, exercise_angina, oldpeak, ST_slope):
    X = np.array([[age, sex, chest_pain_type, resting_bps, cholesterol, fasting_blood_sugar, resting_ecg, max_heart_rate, exercise_angina, oldpeak, ST_slope]]).transpose()
    Z1, A1, Z2, A2 = forward_prop(X)
    print(A2)
    return A2

def predict_heart_disease(age, sex, chest_pain_type, resting_bps, cholesterol, fasting_blood_sugar, resting_ecg, max_heart_rate, exercise_angina, oldpeak, ST_slope):
    output_values = predict_probability(age, sex, chest_pain_type, resting_bps, cholesterol, fasting_blood_sugar, resting_ecg, max_heart_rate, exercise_angina, oldpeak, ST_slope)
    if output_values > .5:
        return "Posible enfermedad cardíaca detectada" 
    else:
        return "Posiblemente sano"    

test_predictions = forward_prop(X_test.transpose())[3] # me interesa solo la capa de salida, A2
test_comparisons = np.equal((test_predictions >= .5).flatten().astype(int), Y_test)
accuracy = sum(test_comparisons.astype(int) / X_test.shape[0])
print("ACCURACY: ", accuracy)

ACCURACY:  0.5289672544080621


In [ ]:
while True:
    col_input = input("Predict heart disease. Input symptoms Age, Sex, Chest pain type, Resting bp/s, Cholesterol, Fasting blood sugar, Resting ec/g, Max heart rate, Excercise angina, Oldpeak, ST_slope: ")
    (age, sex, chest_pain_type, resting_bps, cholesterol, fasting_blood_sugar, resting_ecg, max_heart_rate, exercise_angina, oldpeak, ST_slope) = col_input.split(",")
    print(predict_heart_disease(int(age), int(sex), int(chest_pain_type), int(resting_bps), int(cholesterol), int(fasting_blood_sugar), int(resting_ecg), int(max_heart_rate), int(exercise_angina), int(oldpeak), int(ST_slope)))

In [51]:
pacientes = {
    "Santiago": [40, 1, 2, 140, 289, 0, 0, 172, 0, 0.0, 1],
    "Lorenzo": [49, 0, 3, 160, 180, 0, 0, 156, 0, 1.0, 2],
    "Dante": [37, 1, 2, 130, 283, 0, 1, 98, 0, 0.0, 1],
    "Bruma": [48, 0, 4, 138, 214, 0, 0, 108, 1, 1.5, 2],
    "Martina": [54, 1, 3, 150, 195, 0, 0, 122, 0, 0.0, 1],
    "Alison": [39, 1, 3, 120, 339, 0, 0, 170, 0, 0.0, 1],
    "Guille": [45, 0, 2, 130, 237, 0, 0, 170, 0, 0.0, 1],
    "Luis": [54, 1, 2, 110, 208, 0, 0, 142, 0, 0.0, 1],
    "Cris": [37, 1, 4, 140, 207, 0, 0, 130, 1, 1.5, 2],
    "Tomás": [48, 0, 2, 120, 284, 0, 0, 120, 0, 0.0, 1],
    "Josefina": [58, 1, 2, 136, 164, 0, 1, 99, 1, 2.0, 2]
}

In [53]:
for nombre, valorSintomas in pacientes.items():
    print('-------------------------------')
    print(nombre, valorSintomas)
    [age, sex, chest_pain_type, resting_bps, cholesterol, fasting_blood_sugar, resting_ecg, max_heart_rate, exercise_angina, oldpeak, ST_slope] = valorSintomas
    print(predict_heart_disease(int(age), int(sex), int(chest_pain_type), int(resting_bps), int(cholesterol), int(fasting_blood_sugar), int(resting_ecg), int(max_heart_rate), int(exercise_angina), float(oldpeak), int(ST_slope)))

-------------------------------
Santiago [40, 1, 2, 140, 289, 0, 0, 172, 0, 0, 1]
[[1.]]
Posible enfermedad cardíaca detectada
-------------------------------
Lorenzo [49, 0, 3, 160, 180, 0, 0, 156, 0, 1, 2]
[[1.]]
Posible enfermedad cardíaca detectada
-------------------------------
Dante [37, 1, 2, 130, 283, 0, 1, 98, 0, 0, 1]
[[1.]]
Posible enfermedad cardíaca detectada
-------------------------------
Bruma [48, 0, 4, 138, 214, 0, 0, 108, 1, 1, 2]
[[1.]]
Posible enfermedad cardíaca detectada
-------------------------------
Martina [54, 1, 3, 150, 195, 0, 0, 122, 0, 0, 1]
[[1.]]
Posible enfermedad cardíaca detectada
-------------------------------
Alison [39, 1, 3, 120, 339, 0, 0, 170, 0, 0, 1]
[[1.]]
Posible enfermedad cardíaca detectada
-------------------------------
Guille [45, 0, 2, 130, 237, 0, 0, 170, 0, 0, 1]
[[1.]]
Posible enfermedad cardíaca detectada
-------------------------------
Luis [54, 1, 2, 110, 208, 0, 0, 142, 0, 0, 1]
[[1.]]
Posible enfermedad cardíaca detectada
-